What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames

  Activating project at `~/globtim`


In [2]:
# Constants and Parameters
const n, a, b = 2, 7, 5
const scale_factor = a / b  
f = Deuflhard # Objective function

Deuflhard (generic function with 1 method)

In [3]:
d = 8 # Initial Degree 
SMPL = 200 # Number of samples
TR = test_input(f, 
                dim = n,
                center=[0.0, 0.0],
                GN=SMPL, 
                sample_range=scale_factor
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev);
pol_lege = Constructor(TR, d, basis=:legendre);

current L2-norm: 38.835956522396295
current L2-norm: 24.737307728426945


In [4]:
@polyvar(x[1:n]); # Define polynomial ring 
df_cheb = solve_and_parse(pol_cheb, x, f, TR)
df_lege = solve_and_parse(pol_lege, x, f, TR)


=== Starting MSolve Parser (dimension: 2) ===
Processed 25 points (0.0s)

=== Starting MSolve Parser (dimension: 2) ===
Processed 25 points (0.0s)


Row,x1,x2,z
,Float64,Float64,Float64
1,0.14189,0.14189,4.05672
2,-1.12444,-1.12444,94.2099
3,1.12222,1.12222,91.88
4,0.0428787,0.0428787,4.01373
5,-0.85371,-0.85371,8.56762
6,0.843946,0.843946,8.23594
7,0.0106953,0.0106953,4.00091
8,0.503003,0.503003,2.57837
9,-0.496337,-0.496337,2.54712


In [5]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.05);
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=0.05);

Processing point 1 of 23
Optimization has converged within bounds: ✓
Processing point 2 of 23
Optimization has converged within bounds: ✓
Processing point 3 of 23
Optimization has converged within bounds: ✓
Processing point 4 of 23
Optimization has converged within bounds: ✓
Processing point 5 of 23
Optimization has converged within bounds: ✓
Processing point 6 of 23
Optimization has converged within bounds: ✓
Processing point 7 of 23
Optimization has converged within bounds: ✓
Processing point 8 of 23
Optimization has converged within bounds: ✓
Processing point 9 of 23
Optimization has converged within bounds: ✓
Processing point 10 of 23
Optimization has converged within bounds: ✓
Processing point 11 of 23
Optimization has converged within bounds: ✓
Processing point 12 of 23
Optimization has converged within bounds: ✓
Processing point 13 of 23
Optimization has converged within bounds: ✓
Processing point 14 of 23
Optimization has converged within bounds: ✓
Processing point 15 of 23
Opt

In [6]:
# using CairoMakie
# CairoMakie.activate!

In [7]:
# fig_1 = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb, chebyshev_levels=true)

In [8]:
# fig_2 = cairo_plot_polyapprox_levelset(pol_lege, TR, df_lege, df_min_lege, chebyshev_levels=false)